## Preparation

##### Import Functions

In [4]:
import numpy as np
import pandas as pd
from os import listdir
import pickle
from skimage import io
from matplotlib import pyplot as plt
from skimage.draw import circle, circle_perimeter
from skimage.color import rgb2hsv, rgb2hed, hed2rgb, rgb2gray
from Scripts.LoadImage import LoadImage as LoadImage
from Scripts.Fuzzify import Fuzzify as Fuzzify
from Class.Helper import Helper as Helper
from Scripts.Ki67 import Ki67 as Ki67

##### Functions

In [5]:
def getRecall(markers, mask):
    radius = 3
    hits = markers.apply(lambda marker: mask[int(marker['y'])-radius:int(marker['y']) + radius + 1, int(marker['x'])-radius:int(marker['x']) + radius + 1].any(),axis=1)
    recall = hits.sum() / len(hits)
    return recall

def validateResults(folder_name, low_x, high_x, low_y, high_y, show_images = False):
    path = settings.data_folder_veryfication
    mask = pickle.load(open(settings.backup_folder + "Images/" + "Test_" + folder_name + "_" + settings.class_1 + "_df_results"  + ".p", "rb"))
    mask = mask.sort_index()
    
    if settings.class_1 == "brown":
        searched_class = 1
    else:
        searched_class = 2
    
    mask = np.reshape(np.array(mask["Decision Fuzzy"]), (-1, high_x - low_x))
    
    mask = [np.ones(3) if j == settings.class_1 else np.zeros(3) for row in mask for j in row]
    mask = np.array(mask)
    mask = mask.reshape((-1, high_x - low_x, 3))

    from skimage.morphology import binary_closing
    from skimage.morphology import disk, remove_small_objects
    from skimage.color import rgb2gray
    from skimage import data, io, segmentation, color

    file_path = path + folder_name
    image_raw= io.imread(file_path + "/fragment.png")
    base = io.imread(file_path + "/base.png")
    markers = pd.read_csv(file_path + "/markers.csv")

    k = 3
    markers.loc[:, 'x'] = markers.x.apply(lambda x: np.ceil(x / k))
    markers.loc[:, 'y'] = markers.y.apply(lambda y: np.ceil(y / k))
    markers = markers.drop(markers[((markers.x < low_x) | (markers.x > high_x)) | ((markers.y < low_y) | (markers.y > high_y))].index)
    markers.loc[:, 'x'] = markers.x.apply(lambda x: x - low_x)
    markers.loc[:, 'y'] = markers.y.apply(lambda y: y - low_y)

    markers = markers.loc[markers.type == searched_class]

    if show_images:
        fig, ax = plt.subplots(ncols=5, nrows=1, figsize=(70, 45))
    mask_gray = rgb2gray(mask)
    if show_images:
        ax[0].imshow(mask, cmap=plt.cm.gray)
    mask_gray = binary_closing(mask_gray, disk(1))
    if show_images:
        ax[1].imshow(mask_gray, cmap=plt.cm.gray)
    mask_gray = remove_small_objects(mask_gray, 3)
    if show_images:
        ax[2].imshow(mask_gray, cmap=plt.cm.gray)
    mask_gray = binary_closing(mask_gray, disk(2))
    if show_images:
        ax[3].imshow(mask_gray, cmap=plt.cm.gray)

    mask_gray = mask_gray.astype(bool)
    recall = getRecall(markers, mask_gray)
    
    if show_images:
        for center_y, center_x in zip(markers.y, markers.x):
            circy, circx = circle_perimeter(int(center_y), int(center_x), 0)
            image_raw[circy, circx] = (220, 20, 20)
        
        ax[4].imshow(image_raw, cmap=plt.cm.gray)
        plt.show()
        
    return recall

### Analysis - Input Membership Functions

In [6]:
from Settings.SettingsKi67 import Settings as Settings
import Settings.GeneralSettings as generalSettings

for gauss in [7]:
    for style in ["Gaussian Progressive", "Gaussian Equal"]:
        for adjustment in [-1, -2]:
            for cell_type in ["blue"]:
                generalSettings.gausses = gauss
                generalSettings.style = style
                generalSettings.adjustment_value = adjustment
                generalSettings.class_1 = cell_type
                settings = Settings(generalSettings)        

                # Generate Rules
                loadImage = LoadImage()
                samples_stats, train_stats, test_stats, train_samples = loadImage.worker(settings, test_mode = False)

                fuzzify = Fuzzify()
                changed_decisions, features_number_after_reduct, implicants_number, fuzzify_parameters, execution_time = fuzzify.worker(settings, -1)

                # Validate Results
                for file_name in ["1628-05", "18583-04", "31374-10",]:
                    generalSettings.file_name = file_name
                    settings = Settings(generalSettings)     
                    loadImage = LoadImage()
                    samples_stats, train_stats, test_stats, train_samples = loadImage.worker(settings, test_mode = -1)

                    ki67Test = Ki67(settings, settings.s_function_width, False)
                    ki67Test.noOptymalizationWorker(settings)

                    if file_name == "1628-05":
                        recall_1 = validateResults(settings.file_name, 85, 449, 172, 533, False)
                    elif file_name == "18583-04":
                        recall_2 = validateResults(settings.file_name, 1732, 2011, 583, 995, False)
                    else:
                        recall_3 = validateResults(settings.file_name, 432, 794, 300, 566, False)
                avg_recall = (recall_1 + recall_2 + recall_3) / 3
                helper = Helper()

                fuzzification_data = ["Ki67", settings.style, settings.gausses, settings.adjustment, settings.class_1, settings.features_type, samples_stats, train_stats, test_stats, changed_decisions, round(changed_decisions / train_samples, 2), implicants_number, settings.feature_numbers, features_number_after_reduct, recall_1, recall_2, recall_3, avg_recall, execution_time]
                helper.saveFuzzificationStatsKi67(fuzzification_data)

../Data/Ki67-Train/backgoround.png
../Data/Ki67-Train/blue.png
../Data/Ki67-Train/brown.png
../Data/Ki67-Veryfication/1628-05/fragment.png


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131404/131404 [04:45<00:00, 459.77it/s]


-----------------------------------------------------------------------------------
Time: 299.48659682273865
-----------------------------------------------------------------------------------
../Data/Ki67-Veryfication/18583-04/fragment.png


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114948/114948 [05:20<00:00, 358.35it/s]


-----------------------------------------------------------------------------------
Time: 332.76447796821594
-----------------------------------------------------------------------------------
../Data/Ki67-Veryfication/31374-10/fragment.png


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114948/114948 [05:15<00:00, 363.90it/s]


-----------------------------------------------------------------------------------
Time: 328.2011773586273
-----------------------------------------------------------------------------------
../Data/Ki67-Veryfication/31374-10/fragment.png


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96292/96292 [05:08<00:00, 312.15it/s]


-----------------------------------------------------------------------------------
Time: 318.64808416366577
-----------------------------------------------------------------------------------


,Dataset,Style,Gausses,Adjustment,Searched Class,Features type,Samples,Train s.,Test s.,Changed s.,% changed s.,Implicants,Features,F. after reduct,Recall 1,Recall 2,Recall 3,Average Recall,Execution Time
0,Ki67,Gaussian Progressive,7,Mean,blue,1,96292 (--/--),-- (--/--),96292 (--/--),13,0.0,60,5,5,0.248485,0.348148,0.048387,0.215007,29.965376
1,Ki67,Gaussian Progressive,9,Mean,blue,1,96292 (--/--),-- (--/--),96292 (--/--),14,0.0,84,5,5,0.939394,0.985185,0.698925,0.874501,49.979285
2,Ki67,Gaussian Progressive,11,Mean,blue,1,96292 (--/--),-- (--/--),96292 (--/--),18,0.0,128,5,5,0.812121,0.933333,0.392473,0.712643,94.530699
3,Ki67,Gaussian Progressive,7,Optymalized,blue,1,96292 (--/--),-- (--/--),96292 (--/--),13,0.0,60,5,5,0.248485,0.348148,0.048387,0.215007,31.291521
4,Ki67,Gaussian Progressive,9,Optymalized,blue,1,96292 (--/--),-- (--/--),96292 (--/--),14,0.0,84,5,5,0.939394,0.985185,0.698925,0.874501,50.795949
5,Ki67,Gaussian Equal,7,Mean,blue,1,96292 (--/--),-- (--/--),96292 (--/--),13,0.0,60,5,5,0.248485,0.348148,0.048387,0.215007,31.440280
6,Ki67,Gaussian Progressive,11,Optymalized,blue,1,96292 (--/--),-- (--/--),96292 (--/--),18,0.0,128,5,5,0.812121,0.933333,0.392473,0.712643,95.300114
7,Ki67,Gaussian Equal,9,Mean,blue,1,96292 (--/--),-- (--/--),96292 (--/--),14,0.0,84,5,5,0.939394,0.985185,0.698925,0.874501,51.685072
8,Ki67,Gaussian Equal,7,Optymalized,blue,1,96292 (--/--),-- (--/--),96292 (--/--),13,0.0,60,5,5,0.248485,0.348148,0.048387,0.215007,31.254035
9,Ki67,Gaussian Progressive,7,Mean,brown,1,96292 (--/--),-- (--/--),96292 (--/--),11,0.0,60,5,5,1.000000,1.000000,0.937500,0.979167,29.327340


### Result Validation

In [ ]:
validateResults(settings.file_name, 405, 550, 68, 144, True)